## Setup and Import Libraries

In [1]:
import os
from langchain_groq import ChatGroq
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
llm = ChatGroq(model_name='llama3-8b-8192')

In [5]:
speech = """ Above all, My Lord, we want equal political rights, because without them our disabilities will be permanent. I know this sounds revolutionary 
to the whites in this country, because the majority of voters will be Africans. This makes the white man fear democracy. But this fear cannot be allowed 
to stand in the way of the only solution which will guarantee racial harmony and freedom for all. It is not true that the enfranchisement of all will 
result in racial domination. Political division, based on colour, is entirely artificial and, when it disappears, so will the domination of one colour 
group by another. The ANC has spent half a century fighting against racialism. When it triumphs as it certainly must, it will not change that policy.
This then is what the ANC is fighting. Our struggle is a truly national one. It is a struggle of the African people, inspired by our own suffering and 
our own experience. It is a struggle for the right to live. During my lifetime I have dedicated my life to this struggle of the African people. 
I have fought against white domination, and I have fought against black domination. I have cherished the ideal of a democratic and free society in 
which all persons will live together in harmony and with equal opportunities. It is an ideal for which I hope to live for and to see realised. 
But, My Lord, if it needs be, it is an ideal for which I am prepared to die.
"""

## Text Summarization

In [6]:
chat_message = [
    SystemMessage(content="You are expert in expertise in summarizing speeches"),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech: \n Text: {speech}"),
]

In [8]:
llm.get_num_tokens(speech)

307

In [9]:
llm.invoke(chat_message)

AIMessage(content='Here is a concise summary of the speech:\n\nThe speaker, Nelson Mandela, emphasizes the importance of equal political rights for all, particularly the African people. He acknowledges that this may seem revolutionary to whites, who fear democracy and majority rule. However, he argues that racial harmony and freedom can only be achieved through the enfranchisement of all. The African National Congress (ANC) has spent decades fighting against racialism and will not change its policy when it triumphs. Mandela reiterates his commitment to a democratic and free society where all people can live together in harmony and with equal opportunities, and if necessary, he is willing to die for this ideal.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 337, 'total_tokens': 471, 'completion_time': 0.111666667, 'prompt_time': 0.060416446, 'queue_time': 0.03496677300000001, 'total_time': 0.172083113}, 'model_name': 'llama3-8b-8192

In [10]:
llm.invoke(chat_message).content

'Here is a concise summary of the speech:\n\nThe speaker, Mandela, argues that the African National Congress (ANC) is fighting for equal political rights, which will ensure racial harmony and freedom for all. He asserts that enfranchising all people will not lead to racial domination, but rather the end of political division based on color. The ANC has spent decades combating racialism and will continue to do so. Mandela declares that he has dedicated his life to the struggle for a democratic and free society where all people live together with equal opportunities, and he is willing to die for this ideal.'

## Text Summarization with Prompt Template

In [11]:
generic_template = """ 
Write a summary of the following speech:
Speech: {speech}

Translate the precise summary to {language}
"""

In [12]:
prompt = PromptTemplate(
    input_variables=['speech','language'],
    template=generic_template
)

prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template=' \nWrite a summary of the following speech:\nSpeech: {speech}\n\nTranslate the precise summary to {language}\n')

In [13]:
complete_prompt = prompt.format(
    speech=speech,
    language="Urdu"
)

complete_prompt

' \nWrite a summary of the following speech:\nSpeech:  Above all, My Lord, we want equal political rights, because without them our disabilities will be permanent. I know this sounds revolutionary \nto the whites in this country, because the majority of voters will be Africans. This makes the white man fear democracy. But this fear cannot be allowed \nto stand in the way of the only solution which will guarantee racial harmony and freedom for all. It is not true that the enfranchisement of all will \nresult in racial domination. Political division, based on colour, is entirely artificial and, when it disappears, so will the domination of one colour \ngroup by another. The ANC has spent half a century fighting against racialism. When it triumphs as it certainly must, it will not change that policy.\nThis then is what the ANC is fighting. Our struggle is a truly national one. It is a struggle of the African people, inspired by our own suffering and \nour own experience. It is a struggle 

In [14]:
llm.get_num_tokens(complete_prompt)

332

In [17]:
llm_chain = prompt | llm

In [18]:
summary = llm_chain.invoke({"speech":speech, "language":"Urdu"})
summary

AIMessage(content='Summary:\n\nThe speaker emphasizes the importance of equal political rights for all, particularly for the African people. They argue that without equal rights, their disabilities will remain permanent. The speaker acknowledges that enfranchising all people may seem revolutionary to whites, as it would mean the majority of voters would be Africans. However, they believe that this fear of democracy must not stand in the way of achieving racial harmony and freedom. The African National Congress (ANC) has fought against racialism for half a century and will continue to do so, striving for a democratic and free society where all people can live together with equal opportunities.\n\nUrdu Translation:\n\nمیرے لارڈ، آپے کے لیے سب سے اہم بات یہ ہے کہ تمام لوگوں کو سیاسی حقوق دیے جانے چاہئیں۔ کھللے لوگوں کو یہ سمجھ آئے گا کہ اکثریت ووٹر آفریقہ کے ہوں گے۔ لیکن وہ یہ فریاد نہیں کر سکتے کہ Democracy کا خوف ہے۔ لیکن لڑائی نہیں ہونا چاہئے کہ ہم اپنی فریادوں کو ظاہر کرتے ہیں۔\nاے ان